In [ ]:
import gc

In [1]:
import pandas as pd
import re
import numpy as np
import time
from nltk.stem.porter import PorterStemmer
import numpy as np
import time

import string
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()
stem = stemmer.stem # Makes stemming more accessible

charter_path = '../../charters_full_2015_15_250.pkl'
df_charter = pd.read_pickle(charter_path)
# df_charter['WEBTEXT']=df_charter['WEBTEXT'].fillna('') # turn nan to empty iterable for future convenience
# df_charter = df_charter.iloc[:100] # testing with first 100 schools

dict_path = '/home/jovyan/work/Charter-school-identities/dicts/'
dict_names = ['ess', 'prog', 'rit']
with open(dict_path+dict_names[0]+'_valid.txt') as f: 
    dict_ess = f.read().splitlines()
with open(dict_path+dict_names[1]+'_valid.txt') as f: 
    dict_prog = f.read().splitlines()
with open(dict_path+dict_names[2]+'_valid.txt') as f: 
    dict_rit = f.read().splitlines()

In [2]:
class Page:
    def __init__(self,p):
        self.url = p[0]
        self.boo = p[1]
        self.depth = p[2]
        self.text = p[3]
    def __repr__(self):
        return self.text
    def __eq__(self, other):
        if isinstance(other, Page):
            return self.text == other.text
        else:
            return False
    def __ne__(self, other):
        return (not self.__eq__(other))
    def __hash__(self):
        return hash(self.__repr__())
    
def dict_precalc(keywords):
    small_keywords = []
    large_keywords = []

    for entry in keywords:
        small_keywords.append(entry) if len(re.split('\W+|_', entry)) < 3 else large_keywords.append(entry)

    large_words = [[stem(x) for x in re.split('\W+|_', entry)] for entry in large_keywords] # list words for each large dict entry
    large_lengths = [len(x) for x in large_words]
    large_first_words = [x[0] for x in large_words] # first words of each large entry in dict
    key_words = [[stem(x) for x in re.split('\W+|_', entry)] for entry in keywords]
    return [large_words, large_lengths, large_first_words, key_words]


def dict_count2(key_words, large_words, large_lengths, large_first_words, pages):

    """Returns the hit count with given dictionary on page set.

    pages: set of preprocessed page lists corresponding to an entry of the 'webtext' column
    """
    counts = 0 # number of matches between text_list and custom_dict
    res_length = 0
    # Do dictionary analysis for word chunks of lengths max_entry_length down to 1
    for splitted_phrase in pages:
        for length in range(1, 3):
            if len(splitted_phrase) < length:
                continue # If text chunk is shorter than length of dict entries being matched, there are no matches.
            for i in range(len(splitted_phrase) - length + 1):
                entry = splitted_phrase[i:i+length]
                if entry in key_words:
                    counts += 1
                    res_length += length - 1
        indices = np.transpose(np.nonzero([[word == entry for word in splitted_phrase] for entry in large_first_words]))
        for ind in indices:
            if ind[1] <= (len(splitted_phrase) - large_lengths[ind[0]]) and large_words[ind[0]] == splitted_phrase[ind[1] : ind[1] + large_lengths[ind[0]]]:
                counts += 1
                res_length += large_lengths[ind[0]] - 1
    return counts, res_length

In [4]:
ess_pre = dict_precalc(dict_ess)
prog_pre = dict_precalc(dict_prog)
rit_pre = dict_precalc(dict_rit)
e_res = []
p_res = []
r_res = []
ess_count = []
prog_count = []
rit_count = []
num_words = [] # WARNING: uses original len((re.sub(r'[^\w\s]', '', text)).split()) to count number words
# as opposed to len(re.split('\W+',text))
# this could cause minor inaccuracies (but still better than the latter option in most cases) 
# in hitcount/num_words quotient because of differing interpretations of what a 'word' is
start = time.time()
for i, row in enumerate(df_charter['WEBTEXT'].values):
    pages = set([Page(p) for p in row])
    pages = [[stem(x) for x in re.split('\W+|_', p.text)] for p in pages] # preprocess pages
    num_words.append(sum([len(p) for p in pages]))
    ess_c, ess_res = dict_count2(ess_pre[3], ess_pre[0], ess_pre[1], ess_pre[2], pages)
    prog_c, prog_res = dict_count2(prog_pre[3], prog_pre[0], prog_pre[1], prog_pre[2], pages)
    rit_c, rit_res = dict_count2(rit_pre[3], rit_pre[0], rit_pre[1], rit_pre[2], pages)
    ess_count.append(ess_c)
    prog_count.append(prog_c)
    rit_count.append(rit_c)
    e_res.append(ess_res)
    p_res.append(prog_res)
    r_res.append(rit_res)
    if i%100 == 0:
        end = time.time()
        print('Time Elapsed:{:f}, Percent Complete:{:f}'.format(end - start,i*100/len(df_charter)))
df_charter['PROG_COUNT'] = pd.Series(prog_count, index=df_charter.index)
df_charter['RIT_COUNT'] = pd.Series(rit_count, index=df_charter.index)
df_charter['ESS_COUNT'] = pd.Series(ess_count, index=df_charter.index)
df_charter['NUMWORDS'] = pd.Series(num_words, index=df_charter.index)

Time Elapsed:0.046479, Percent Complete:0.000000
Time Elapsed:24.980927, Percent Complete:0.867980
Time Elapsed:27.147228, Percent Complete:1.735960
Time Elapsed:101.041536, Percent Complete:2.603941
Time Elapsed:317.712899, Percent Complete:3.471921
Time Elapsed:366.894080, Percent Complete:4.339901
Time Elapsed:495.462567, Percent Complete:5.207881
Time Elapsed:597.561799, Percent Complete:6.075861
Time Elapsed:701.807601, Percent Complete:6.943842
Time Elapsed:868.792875, Percent Complete:7.811822
Time Elapsed:970.750667, Percent Complete:8.679802
Time Elapsed:1057.899517, Percent Complete:9.547782
Time Elapsed:1121.714159, Percent Complete:10.415763
Time Elapsed:1193.439125, Percent Complete:11.283743
Time Elapsed:1402.608823, Percent Complete:12.151723
Time Elapsed:1647.596335, Percent Complete:13.019703
Time Elapsed:1716.613921, Percent Complete:13.887683
Time Elapsed:1790.622555, Percent Complete:14.755664
Time Elapsed:1797.613959, Percent Complete:15.623644
Time Elapsed:1889.63

In [5]:
df_charter.loc[:,'ESS_STR'] = np.log10(np.array(ess_count)/(np.array(num_words) - np.array(e_res)))
df_charter.loc[:,'PROG_STR'] = np.log10(np.array(prog_count)/(np.array(num_words) - np.array(p_res)))
df_charter.loc[:,'RIT_STR'] = np.log10(np.array(rit_count)/(np.array(num_words) - np.array(r_res)))
df_charter.replace([np.inf, -np.inf], np.nan, inplace = True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [1]:
df_charter.to_pickle('../../charters_full_2015_250.pkl')

NameError: name 'df_charter' is not defined

In [ ]:
df_charter.iloc[8000:].to_pickle('charters_full_250_3.pkl')

In [ ]:
df_charter.iloc[4000:8000].to_pickle('charters_full_250_2.pkl')